In [ ]:
import requests
import bs4
import pandas as pd

In [ ]:
def scrape_listings(url):
    # Open the URL
    r = requests.get(url)
    # Parse the HTML
    soup = bs4.BeautifulSoup(r.text, 'html.parser')

    # Find all td-elements with class "horizontal-half-padder-minus pr"
    listings = soup.find_all('td', {'horizontal-half-padder-minus pr'})
    # Iterate over the listings
    rows = []
    for element in listings:
        row = {}
        # Find the a-element in element
        row['url'] = "https://www.arabam.com/" + element.find('a').get('href')
        rows.append(row)
    return pd.DataFrame(rows)

Dies kann verwendet werden, um Auflistungs-URLs von Autos auf der Webseite „www.arabam.com“ zu extrahieren. Es nutzt die Bibliotheken Pandas, Requests und BeautifulSoup aus bs4 (BeautifulSoup4). Die Funktion akzeptiert eine URL als Eingabe, lädt den Inhalt der Seite, analysiert das HTML, sucht nach bestimmten -Elementen mit einer spezifischen Klasse und wechselt dann die URLs von Listings aus den „a“ Elementen in einem Data Frame, bevor es diesen DataFrame zurückgibt .

In [ ]:
df = scrape_listings("https://www.arabam.com/ikinci-el/otomobil/skoda?take=50")
df

,url
0,https://www.arabam.com//ilan/galeriden-satilik...
1,https://www.arabam.com//ilan/galeriden-satilik...
2,https://www.arabam.com//ilan/galeriden-satilik...
3,https://www.arabam.com//ilan/galeriden-satilik...
4,https://www.arabam.com//ilan/galeriden-satilik...
5,https://www.arabam.com//ilan/galeriden-satilik...
6,https://www.arabam.com//ilan/galeriden-satilik...
7,https://www.arabam.com//ilan/galeriden-satilik...
8,https://www.arabam.com//ilan/galeriden-satilik...
9,https://www.arabam.com//ilan/galeriden-satilik...


Das der Variablen df zugewiesene Ergebnis stellt einen Pandas DataFrame dar, der die URLs dieser Autoanzeigen enthält. Wenn wir hier die df-Variable drucken, sehen wir eine Tabelle mit den URLs der relevanten Anzeigen.

In [ ]:
def scrape_details(df):
    # Iterate over the rows of df
    for i in range(len(df)):
        # Open the URL
        r = requests.get(df['url'].iloc[i])
        # Parse the HTML
        soup = bs4.BeautifulSoup(r.text, 'html.parser')
        # Find the div-element with class 'product-properties' in soup
        pricing = soup.find('div', {'class': 'product-properties'})
        # Find all sd-cell-row-elements in pricing
        for cell in pricing.find_all('div', {'class': 'property-item'}):
            # Find all sd-cell-col-elements in cell
            cols = cell.find('div', {'class': 'property-key'})
            rows = cell.find_all('div', {'class': 'property-value'})
            # Add a new column to the df if not existing and set the i-th value for the column
            if cols and cols.text.strip() not in df.columns:
                df[cols.text.strip()] = None


            if rows and cols:
                    df.loc[i, cols.text.strip()] = rows[0].text.strip() if rows else None
        for cell in pricing.find_all('div', {'class': 'product-properties--sticky'}):
            for cell2 in cell.find_all('div', {'class': 'product-price-container'}):

                cols = cell2.find('div', {'class': 'property-key'})
                rows = cell2.find_all('div', {'class': 'product-price'})
                # Add a new column to the df if not existing and set the i-th value for the column
                if cols and cols.text.strip() not in df.columns:
                    df[cols.text.strip()] = None


                if rows and cols:
                        df.loc[i, cols.text.strip()] = rows[0].text.strip() if rows else None

    return df.drop(["İlan No", "Kimden", "url", "İlan Tarihi", "Takasa Uygun"], axis=1)

#"İlan No" : „Anzeigennummer“
#"Kimden" : „Zu verkaufen von“
#"url" : „URL“
#"İlan Tarihi" : „Anzeigendatum“
#"Takasa Uygun" : „Für den Handel verfügbar“

Diese Funktion macht folgendes:
Öffnet die URL in der Spalte „URL“ des Datenrahmens für jede Zeile und verwendet den von der Funktion „scrape_listings“ erstellten Datenrahmen, um die Auflistungsdetails abzurufen.
Analysiert das HTML der Website mit BeautifulSoup, um bestimmte Elemente zu finden.
Extrahiert Informationen aus diesen Elementen, insbesondere aus div-Tags mit verschiedenen Klassen wie 'product-properties', 'property-item', 'product-properties--sticky' usw.
Fügt die extrahierten Informationen als neue Spalten dem Data Frame hinzu.
Entfernt bestimmte Spalten wie "İlan No": „Anzeigennummer“ , "Kimden": „Zu verkaufen von“, "url", "İlan Tarihi": „Anzeigendatum“ und "Takasa Uygun": „Für den Handel verfügbar“ aus dem DataFrame.
Gibt den bearbeiteten DataFrame zurück, der nun die erfassten Informationen der Webseite in strukturierter Form enthält.


<img src="product-properties.png" />

Zunächst sehen wir im Allgemeinen den Teil „product-properties“ eines solchen Autos


<img src="property-item.png" />

So behandeln wir eine einzelne Eigenschaft des Autos mit der div „property-item“.

<img src="property-key-value.png" />

Wenn wir diesen Abschnitt öffnen, sehen wir das Div, in dem ein „property-Item“, also eine Eigenschaft des Fahrzeugs, ein „key“ ist und diese keys „value“ sind.

<img src="product-price-container.png" />

Da die Preise des Fahrzeugs nicht in „property-item“, sondern in einem anderen div angegeben sind, ziehen wir die Preise des Fahrzeugs aus diesem im Bild gezeigten Pfad.

<img src="drop.png" />

Wir finden die „key“ aus dem „property-item“ der unnötigen Merkmale, die wir nicht verwenden und aus dem DataFrame entfernen. Anschließend entfernen wir diese unnötigen Daten in der von uns vorbereiteten Funktion scrape_details.
Diese Merkmale sind(mit ihren deutschen Entsprechungen):
"İlan No" : „Anzeigennummer“
"Kimden" : „Zu verkaufen von“
"url" : „URL“
"İlan Tarihi" : „Anzeigendatum“
"Takasa Uygun" : „Für den Handel verfügbar“

In [ ]:
df = scrape_details(df)
df

In [ ]:
def scrape_all(pages):
    url = 'https://www.arabam.com/ikinci-el/otomobil/skoda?take=50&page='
    df = pd.DataFrame()
    for page in pages:
        df = pd.concat([df, scrape_listings(url + str(page))], ignore_index=True)
    df = scrape_details(df)
    return df

Die Funktion macht folgendes:
Die Funktion erstellt eine Basis-URL für die Autoverkaufsseite ('https://www.arabam.com/ikinci-el/otomobil/skoda?take=50&page=') mit einem Platzhalter für die Seitenzahl.
Es initialisiert ein leeres Pandas DataFrame (df), um die gesammelten Daten zu speichern.
Für jede Seitenzahl in der übergebenen Liste pages wird scrape_listings aufgerufen, um Listings von der entsprechenden Seite zu extrahieren.
Die gesammelten Listings werden dem DataFrame df hinzugefügt.
Anschließend wird die Funktion scrape_details aufgerufen, um detailliertere Informationen zu den Listings zu sammeln und dem DataFrame hinzuzufügen.
Schließlich gibt die Funktion den bearbeiteten DataFrame zurück, der alle gesammelten und strukturierten Informationen von den angegebenen Seiten enthält.
In diesem Fall  wurde die Marke “Skoda” extrahiert aber die Marken Audi, BMW,  Citroen, Fiat, Ford, Honda, Hyundai,  Mercedes - Benz, Opel,  Peugeot,  Seat, Skoda, Toyota, Volkswagen für das Modell benutzt.


In [ ]:
df_large = scrape_all(range(1, 50))
df_large